# Playing with rasterio and fiona

#### Variable declarations

sample_points_filepath – path to sample points shapefile <br />
DEM_filepath – path to DEM raster <br />
elevation_filepath – path to export excel file containing elevation values for each sample site

In [2]:
sample_points_filepath = ""

In [4]:
DEM_filepath = ""

In [5]:
elevation_filepath = ""

#### Import statements

In [150]:
import rasterio
import fiona
import pandas
import numpy
from pyproj import Proj, transform
from fiona.crs import from_epsg

In [226]:
with fiona.open(sample_points_filepath, 'r') as source_points:
    points = [f['geometry']['coordinates'] for f in source_points]
    
    original = Proj(source_points.crs)
    destination = Proj(from_epsg(4326))
    #destination = Proj(' +proj=latlong +ellps=bessel')
    
    with rasterio.drivers():
        with rasterio.open(DEM_filepath) as source_dem:
            s = source_dem.sample(points)
            elevs = numpy.array([n[0] for n in s])
            source_dem.close
    source_points.close

## Transform points

In [227]:
points_projected = []
for p in points:
    x, y = p
    lat, long = transform(original, destination, x, y)
    points_projected.append((long,lat))

In [228]:
points_projected_pd = pandas.DataFrame(points_projected, columns=["lat", "long"])

In [229]:
with fiona.open(sample_points_filepath, 'r') as source_points:
    names = numpy.array([p['properties']['NAME'] for p in source_points])
    IDs = numpy.array([p['properties']['ID'] for p in source_points])
    
    source_points.close

In [230]:
elevs_names = [{"ID":IDs[i],"elevation":elevs[i], "name":names[i], "latitude":points_projected[i][0], "longitude":points_projected[i][1]} for i in range(len(elevs))]

In [231]:
elevs_pd = pandas.DataFrame(elevs_names)

In [232]:
elevs_pd

,ID,elevation,latitude,longitude,name
0,2,665,47.267357,11.305684,Hofwald
1,103,768,47.270798,11.334081,Kerschbuchhof
2,4,725,47.270846,11.336290,Kranebitten
3,25,717,47.272745,11.358701,Buzihütte
4,27,698,47.272116,11.368856,Sadrach
5,7,639,47.267641,11.366170,Höttinger Rain
6,102,687,47.270875,11.374611,Schießstand
7,101,607,47.268002,11.378812,Sternwarte
8,22,906,47.288550,11.398594,Hungerburg
9,104,906,47.288532,11.398566,Hungerburg


In [233]:
elevs_pd.to_excel(elevation_filepath)